This is an implementation of depolying CNN gensture reconition model to edge device (e.g. Sony Spresense)

Step Overview:
1. Conver Pytorch Model to Onnx Model
2. Conver Onnx Model to Tensorflow Model
3. Quantizte the Tensorflow Model and convert it to Tensorflow Lite Model
4. Using Edge Impuls to generate code for Sony Spresense

# Conver Pytorch Model to Onnx Model

In [11]:
import torch
import torchvision
from models.mobilenetv2_no_bn import MobileNetV2
dummy_input = torch.randn(1, 1, 8, 24, device="cuda")
model = MobileNetV2(num_classes=8, input_layer=1, model_width=0.2).cuda()
model.load_state_dict(torch.load("pretrain_model/pretrain_model_no_bn/MobilenetV2_Params@797.096K_MAC@4.555M_Acc@97.912.pt"), strict=False)
model.eval()

torch.onnx.export(model, dummy_input, "pretrain_model/onnx_model/MobilenetV2.onnx", verbose=True)

Exported graph: graph(%input.1 : Float(1, 1, 8, 24, strides=[192, 192, 24, 1], requires_grad=0, device=cuda:0),
      %conv1.weight : Float(32, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %layers.0.conv1.weight : Float(32, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %layers.0.conv2.weight : Float(32, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %layers.0.conv3.weight : Float(16, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %layers.0.shortcut.0.weight : Float(16, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %layers.1.conv1.weight : Float(16, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cuda:0),
      %layers.1.conv2.weight : Float(16, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %layers.1.conv3.weight : Float(24, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cuda:0),
      %layers.1.shortcut.0.weight : Float(

# Conver Onnx Model to Tensorflow Model

In [12]:
# Install library
%cd onnx-tensorflow


/home/studenta/Documents/CNN-HD-sEMG-Classifier/onnx-tensorflow


In [ ]:
# Install library
!pip install -e .

In [14]:


import onnx

onnx_model = onnx.load("../pretrain_model/onnx_model/MobilenetV2.onnx")

from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model)
tf_rep.export_graph("../pretrain_model/tf_model")


2023-05-28 00:42:57.297687: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-28 00:42:57.317376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 00:42:57.639282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/studenta/anaconda3/envs/sEMG/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until

INFO:tensorflow:Assets written to: ../pretrain_model/tf_model/assets


INFO:tensorflow:Assets written to: ../pretrain_model/tf_model/assets


# Quantization aware training

In [38]:
import tensorflow_model_optimization as tfmot
# quantize_model = tfmot.quantization.keras.quantize_model
model = tf.keras.models.load_model('../pretrain_model/tf_model')
# model = tf.saved_model.load("../pretrain_model/tf_model")
# q_aware_model = quantize_model(model)
# q_aware_model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# q_aware_model.summary()
model.summary()

AttributeError: '_UserObject' object has no attribute 'summary'

# Quantizte the Tensorflow Model and convert it to Tensorflow Lite Model

In [ ]:
# Prepare representive data from quantization
import scipy.io
input_data = scipy.io.loadmat("data/sEMG_Test_Label_hand_close.mat")['Data']
import numpy as np
input_data = np.array(input_data)
input_data = input_data.reshape(-1, 1, 8, 24)

# Normalize the input data by subtracting the mean and dividing by the standard deviation
input_data = (input_data - input_data.mean()) / input_data.std()
np.save("representive_data",input_data)

In [15]:
import numpy as np
def representative_dataset():
    data = np.load("../representive_data.npy")
    for i in range(1):
        temp_data = data[i]
        temp_data = temp_data.reshape(1,1,8,24)
        yield [temp_data.astype(np.float32)]

In [17]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("../pretrain_model/tf_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_model = converter.convert()

# Save the model
with open("../pretrain_model/tf_lite_model/mobilenetv2.tflite", 'wb') as f:
    f.write(tflite_model)

2023-05-28 00:45:06.132373: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-28 00:45:06.132388: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-28 00:45:06.132731: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../pretrain_model/tf_model
2023-05-28 00:45:06.134298: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-28 00:45:06.134331: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../pretrain_model/tf_model
2023-05-28 00:45:06.141600: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-05-28 00:45:06.142955: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-28 00:45:06.182579: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: ../p

# Simulate Model Accuracy

In [21]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="../pretrain_model/tf_lite_model/mobilenetv2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)
input_data = np.load(f"../representive_data.npy")
ori_input_data = input_data.astype(np.float32)
correct = 0
print("Total Sample Size:",ori_input_data.shape[0])
for i in range(input_data.shape[0]):
    input_data = np.expand_dims(ori_input_data[i], 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # get_tensor() returns a copy of the tensor data
    # use tensor() in order to get a pointer to the tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if np.argmax(output_data) == 0:
        correct += 1
print("Prediction Correct Size:",correct) #Total:30720
print("Accuracy",round(correct/int(ori_input_data.shape[0]),2))

Total Sample Size: 30720
Prediction Correct Size: 24770
Accuracy 0.81


In [4]:
import scipy.io
input_data = scipy.io.loadmat("data/Training_Trimmed/001-001-001.mat")['Data']
import numpy as np
input_data = np.array(input_data)
input_data = input_data.reshape(-1, 1, 8, 24)

# Normalize the input data by subtracting the mean and dividing by the standard deviation
input_data = (input_data - input_data.mean()) / input_data.std()
# np.save("representive_data8",input_data)
for i in input_data[0].reshape(-1):
    print(i,end=",")

-0.9778049,-2.2867649,-0.008407739,-1.0928181,-1.1311558,-0.9558976,-1.0544803,-0.80802345,0.16137369,0.21066505,0.14494322,-0.96137446,-0.7861162,-1.1202022,-0.96137446,-0.944944,-1.0982949,-0.0248382,0.15589687,-0.75325525,-0.90112936,0.17232732,-1.0928181,-0.63276523,-0.93946713,-1.0763876,-0.7696857,-1.0544803,-0.944944,-1.2133081,-1.065434,-0.5615666,-1.2845068,-1.065434,-1.2899836,-0.9778049,-1.0325731,-1.2406923,-0.7039639,-0.46298382,-1.3776127,-1.487149,-1.185924,-1.1640167,-0.8846989,-0.72587115,-0.6218116,-0.42464608,-1.6624073,-1.6076392,-1.3885663,-0.87922215,-0.9120831,-0.58347386,-0.42464608,-0.24938783,-1.8486191,-1.9910165,-1.2242619,-0.9285135,-0.5067984,-0.2658183,-0.16175869,-0.304156,-1.15854,-1.0928181,-1.1694936,-1.185924,-1.1914009,-1.0928181,-1.1366327,-1.2461691,-1.2625995,-1.1202022,-1.1749704,-1.1475863,-1.0709108,-1.0763876,-1.1640167,-0.99971217,-1.1147254,-1.0928181,-0.95042074,-1.125679,-1.0928181,-1.0982949,-0.8956526,-1.0380499,-1.0818645,-1.15854,-1.0

In [10]:

# import necessary libraries
import pandas as pd
import numpy as np
 
# create a dummy array
arr = np.load("representive_data.npy")
 
# display the array
# print(arr)
 
# convert array into dataframe
DF = pd.DataFrame(arr.reshape(-1,192))
DF[len(DF.columns)] = 0

# save the dataframe as a csv file
DF.to_csv("representive_data.csv")